# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

In [ ]:
!pip install catboost

Import packages, classifiers and etc.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

Import VAEOversampler.

In [ ]:
from VAEOversampler import VAEOversampler

## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [ ]:
from imblearn.datasets import fetch_datasets

dset_name = 'satimage'
dset = fetch_datasets()[dset_name]
dset.data.shape

print(sorted(Counter(dset.target).items()))

[(-1, 5809), (1, 626)]


In [ ]:
dset.data

array([[ 92., 115., 120., ..., 107., 113.,  87.],
       [ 84., 102., 106., ...,  99., 104.,  79.],
       [ 84., 102., 102., ...,  99., 104.,  79.],
       ...,
       [ 56.,  68.,  91., ...,  83.,  92.,  74.],
       [ 56.,  68.,  87., ...,  83.,  92.,  70.],
       [ 60.,  71.,  91., ...,  79., 108.,  92.]])

In [ ]:
X = pd.DataFrame(dset.data)
y = dset.target

In [ ]:
y[y == -1] = 0

In [ ]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 5809, 1: 626})
Ratio-> 9.3 : 1


We split data into train and test partitions.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [ ]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res  
    
    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1)
    
    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [ ]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))
    

## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [ ]:
vae_sampler = VAEOversampler(epochs=50,
                              intermediate_dim=512,
                              batch_size=64,
                              rescale=True,
                              random_state=42,
                              verbose=False)
Xres, yres = vae_sampler.fit_resample(X_train, y_train, validation_data=[X_test, y_test])

130/130 [==============================] - 0s 2ms/step


In [ ]:
print('Resampled dataset shape %s' % Counter(yres))
print('Ratio->  1 :', round(Counter(yres)[1]/Counter(yres)[0], 1))

Resampled dataset shape Counter({1.0: 4647, 0.0: 4647})
Ratio->  1 : 1.0


In [ ]:
train_val(Xres, yres, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1141   21]
 [  49   76]]

TNR: 0.98193
TPR: 0.608
TNRxTPR: 0.59701
G-mean: 0.77267
__________________________________________________
Classifier:  LGBMClassifier
[[1140   22]
 [  41   84]]

TNR: 0.98107
TPR: 0.672
TNRxTPR: 0.65928
G-mean: 0.81196
__________________________________________________
Classifier:  XGBClassifier
[[1139   23]
 [  42   83]]

TNR: 0.98021
TPR: 0.664
TNRxTPR: 0.65086
G-mean: 0.80676
__________________________________________________
Classifier:  BaggingClassifier
[[1144   18]
 [  62   63]]

TNR: 0.98451
TPR: 0.504
TNRxTPR: 0.49619
G-mean: 0.70441
__________________________________________________
Classifier:  RandomForestClassifier
[[1149   13]
 [  58   67]]

TNR: 0.98881
TPR: 0.536
TNRxTPR: 0.53
G-mean: 0.72801
__________________________________________________
Ensemble predictions (majority voting):
[[1142   20]
 [  49   76]]

TNR: 0.98279
TPR: 0.608
TNRxTPR: 0.59754
G-mean: 0

## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [ ]:
Xres, yres = vae_sampler.resample(X_train, y_train, sampling_strategy=.6)

78/78 [==============================] - 1s 6ms/step


In [ ]:
# RUS

X_eq, y_eq = RUS(Xres, yres, frac=.16)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 2988, 0.0: 744})
Ratio->  1 : 4.0


In [ ]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1031  131]
 [  18  107]]

TNR: 0.88726
TPR: 0.856
TNRxTPR: 0.7595
G-mean: 0.87149
__________________________________________________
Classifier:  LGBMClassifier
[[1021  141]
 [  12  113]]

TNR: 0.87866
TPR: 0.904
TNRxTPR: 0.79431
G-mean: 0.89124
__________________________________________________
Classifier:  XGBClassifier
[[1027  135]
 [   7  118]]

TNR: 0.88382
TPR: 0.944
TNRxTPR: 0.83433
G-mean: 0.91342
__________________________________________________
Classifier:  BaggingClassifier
[[1039  123]
 [  24  101]]

TNR: 0.89415
TPR: 0.808
TNRxTPR: 0.72247
G-mean: 0.84998
__________________________________________________
Classifier:  RandomForestClassifier
[[1045  117]
 [  16  109]]

TNR: 0.89931
TPR: 0.872
TNRxTPR: 0.7842
G-mean: 0.88555
__________________________________________________
Ensemble predictions (majority voting):
[[1033  129]
 [  13  112]]

TNR: 0.88898
TPR: 0.896
TNRxTPR: 0.79653
G-mean: 

XGBClassifier  
  - G-mean: **0.91342**  


https://imbalanced-learn.org/stable/auto_examples/ensemble/plot_comparison_ensemble_classifier.html#sphx-glr-auto-examples-ensemble-plot-comparison-ensemble-classifier-py  

In this web we can compare our results on 'satimage' dataset with some balanced versions of classical algorithms like: **BalancedBaggingClassifier**, **BalancedRandomForestClassifier**.  


https://imbalanced-learn.org/stable/combine.html  
Here we find two versions of SMOTE that combines over- and under-sampling: **SMOTEENN** and **SMOTETomek**.

## Cross Validation  

In [ ]:
g_mean = make_scorer(geometric_mean_score)

clf = XGBClassifier(random_state=42)

cv_results = cross_validate(clf, X_eq, y_eq, scoring=g_mean, cv=10,
                            return_estimator=True, n_jobs=-1)

print(f"G-mean CV: {cv_results['test_score'].mean():.3f} (+/-{cv_results['test_score'].std():.3f})")

G-mean CV: 0.928 (+/-0.014)


In [ ]:
scores = []
for fold_id, cv_model in enumerate(cv_results['estimator']):
    scores.append(geometric_mean_score(y_test, cv_model.predict(X_test)))
    
print(f"G-mean CV (test): {np.mean(scores):.3f} (+/-{np.std(scores):.3f})")

G-mean CV (test): 0.883 (+/-0.014)


Classification report  

In [ ]:
y_pred = XGBClassifier(random_state=42).fit(X_eq, y_eq).predict(X_test)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.88      0.94      0.94      0.91      0.83      1162
          1       0.47      0.94      0.88      0.62      0.91      0.84       125

avg / total       0.94      0.89      0.94      0.91      0.91      0.83      1287



## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
